In [254]:

import requests
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import json
from pandas.io.json import json_normalize


from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

%load_ext autoreload
%autoreload 2

import seaborn as sns
pd.reset_option('display.max_rows')

import glob

from  get_diff import calculate_event_differences
from clean_preprocess import preprocess, specify_interest_features, remove_duplicates, replace_nan_with_zero

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:

# Ruta al directorio que contiene los archivos CSV
ruta_directorio = "../data/Match_Diff/"

# Obtener la lista de todos los archivos CSV en el directorio
archivos_csv = glob.glob(ruta_directorio + "*.csv")

# Inicializar una lista para almacenar los DataFrames de cada archivo
dfs = []

# Leer y combinar los archivos CSV en un solo DataFrame
for archivo in archivos_csv:
    df = pd.read_csv(archivo)
    dfs.append(df)

df_combinado = pd.concat(dfs, ignore_index=True)

df = df_combinado         


In [221]:
df_filtered = specify_interest_features(df)
df_filtered



['monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'monsterType_AIR_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'towerType_OUTER_TURRET',
 'towerType_INNER_TURRET',
 'towerType_BASE_TURRET',
 'towerType_NEXUS_TURRET',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'killType_KILL_ACE',
 'minionsKilled',
 'totalGold']

In [275]:
from sklearn.preprocessing import StandardScaler

df_train_preprocessed, y_train = preprocess(df_filtered)
X_train = pd.DataFrame(df_train_preprocessed, columns=df_filtered.drop("target", axis=1).columns)

# Reemplazar valores faltantes por ceros en X_train
X_train = replace_nan_with_zero(X_train)

# Escalar los datos utilizando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Calcular los puntajes en el conjunto de prueba
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Duplicates dropped: 43
Accuracy: 0.7016797556719022


In [299]:
# Obtener los nombres de las columnas antes de escalar los datos
column_names = df_filtered.drop("target", axis=1).columns.tolist()
column_names

['monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'monsterType_AIR_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'towerType_OUTER_TURRET',
 'towerType_INNER_TURRET',
 'towerType_BASE_TURRET',
 'towerType_NEXUS_TURRET',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'killType_KILL_ACE',
 'minionsKilled',
 'totalGold']

In [279]:
minute = 10

#Predecir el resultado para un json
# Lee el archivo JSON


json_file = pd.read_json('../data/IRON/Timeline/Partidas/LA2_1003328463_timeline.json')
data = pd.DataFrame(json_file["info"]["frames"][minute]["participantFrames"])
    
data = data.transpose().drop(columns=["currentGold", "goldPerSecond", "participantId", "position", "timeEnemySpentControlled", "level"])
data.reset_index(drop=True, inplace=True)

In [262]:
data

,championStats,damageStats,jungleMinionsKilled,minionsKilled,totalGold,xp
0,"{'abilityPower': 0, 'armor': 66, 'armorPen': 0...","{'magicDamageDone': 0, 'magicDamageDoneToChamp...",0,41,2284,3980
1,"{'abilityPower': 0, 'armor': 42, 'armorPen': 0...","{'magicDamageDone': 0, 'magicDamageDoneToChamp...",0,0,1500,0
2,"{'abilityPower': 51, 'armor': 50, 'armorPen': ...","{'magicDamageDone': 16698, 'magicDamageDoneToC...",0,57,2962,4399
3,"{'abilityPower': 0, 'armor': 38, 'armorPen': 0...","{'magicDamageDone': 176, 'magicDamageDoneToCha...",0,38,2373,2200
4,"{'abilityPower': 57, 'armor': 34, 'armorPen': ...","{'magicDamageDone': 5969, 'magicDamageDoneToCh...",0,4,2370,2718
5,"{'abilityPower': 64, 'armor': 56, 'armorPen': ...","{'magicDamageDone': 9846, 'magicDamageDoneToCh...",0,37,2763,4518
6,"{'abilityPower': 0, 'armor': 68, 'armorPen': 0...","{'magicDamageDone': 11370, 'magicDamageDoneToC...",52,3,3169,3426
7,"{'abilityPower': 0, 'armor': 52, 'armorPen': 0...","{'magicDamageDone': 2228, 'magicDamageDoneToCh...",0,46,2559,3802
8,"{'abilityPower': 0, 'armor': 50, 'armorPen': 0...","{'magicDamageDone': 124, 'magicDamageDoneToCha...",0,56,3918,3584
9,"{'abilityPower': 0, 'armor': 47, 'armorPen': 0...","{'magicDamageDone': 0, 'magicDamageDoneToChamp...",0,8,3420,2450


In [280]:
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"] 

minute = 10

folder_path = "../data/IRON/Timeline/Prueba/" #Change to your folder path


In [281]:
#PARTICIPANT FRAMES

def part_frames(json_file):
    
    data = pd.DataFrame(json_file["info"]["frames"][minute]["participantFrames"])
    
    data = data.transpose().drop(columns=["currentGold", "goldPerSecond", "participantId", "position", "timeEnemySpentControlled", "level"])
    data.reset_index(drop=True, inplace=True)

    damage_stats = pd.json_normalize(data["damageStats"])
    champion_stats = pd.json_normalize(data["championStats"])

    pf_all = pd.concat([data, damage_stats, champion_stats], axis=1).drop(columns=["championStats","damageStats"])
    pf_all = pf_all.astype(int)

    conditions = [
            (pf_all.index >= 0) & (pf_all.index <= 4),
            (pf_all.index >= 5) & (pf_all.index <= 9)
        ]
    values = [100, 200]

    pf_all['team'] = np.select(conditions, values, default=0)

    pf_all = pf_all.groupby("team").sum().reset_index()

    return pf_all


#EVENTS & TARGET

def find_indices(data):
    monster_indices = []
    for index, item in enumerate(data):
        if item.get('type') in look_events:
            monster_indices.append(index)
    return monster_indices

def get_events(json_file):
    elements_filtered = []
    for i in range(minute + 1):
        try:
            events = json_file["info"]["frames"][i]["events"]
        except IndexError:
            break
        monster_indices = find_indices(events)
        if monster_indices:
            new_elements = [events[e] for e in monster_indices]
            elements_filtered.extend(new_elements)

    events_filtered = pd.DataFrame(elements_filtered)
    return events_filtered


def transform_events(json_file):
    required_columns = ["killerId", "type", "killType", "monsterSubType", "monsterType", "towerType"]

    events = get_events(json_file)

    missing_columns = [col for col in required_columns if col not in events.columns]

    for column in missing_columns:
        events[column] = np.nan

    events = events[required_columns]
    events = events[events["killerId"] != 0]

    events.loc[events["monsterType"] == "DRAGON", "monsterType"] = events.loc[events["monsterType"] == "DRAGON", "monsterSubType"]

    events = events.drop(columns=["monsterSubType"])

    events["kills"] = (events["type"] == "CHAMPION_KILL").astype(int)

    one_hot_encoded = pd.get_dummies(events[['killType', "monsterType", "towerType"]])

    events_encoded = pd.concat([events[['killerId', "kills"]], one_hot_encoded], axis=1)
    
    if 'killType' in events_encoded.columns:
        events_encoded.drop(columns=['killType'], inplace=True)
    if 'monsterType' in events_encoded.columns:
        events_encoded.drop(columns=['monsterType'], inplace=True)
    if 'towerType' in events_encoded.columns:
        events_encoded.drop(columns=['towerType'], inplace=True)
    
    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    events_encoded["killerId"] = events_encoded["killerId"] - 1

    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    conditions = [
        (events_encoded['killerId'] >= 0) & (events_encoded['killerId'] <= 4),
        (events_encoded['killerId'] >= 5) & (events_encoded['killerId'] <= 9)
    ]
    values = [100, 200]

    events_encoded['team'] = np.select(conditions, values, default=0)

    events_encoded = events_encoded.groupby("team").sum().reset_index().drop(columns=["killerId"])
    
    return events_encoded


def merge_dfs(json_file):
    events = transform_events(json_file)
    frames = part_frames(json_file)
    
    dfs = frames.merge(events,how="left",on="team")
    
    dfs["matchId"] = json_file["metadata"]["matchId"]
    
    last_event = json_file["info"]["frames"][-1]["events"][-1]
    dfs["target"] = dfs["team"].apply(lambda x: 1 if x == last_event.get("winningTeam") else 0)
    
    dfs.drop(columns=["team"],inplace=True)
    
    return dfs



In [282]:
file_path = os.path.join(folder_path, filename)
        
json_file = pd.read_json(file_path)
json_file

,metadata,info
dataVersion,2,NaN
matchId,LA2_1304858815,NaN
participants,[g5H5VAZmGEyLO3MC7trd6K92DQxDCvKfAULYMeQQg0nzT...,"[{'participantId': 1, 'puuid': 'g5H5VAZmGEyLO3..."
frameInterval,NaN,60000
frames,NaN,"[{'events': [{'realTimestamp': 1686264389778, ..."
gameId,NaN,1304858815


In [283]:
#LOOP PARA TODOS LOS JSON


all_events = None


for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        json_file = pd.read_json(file_path)
           
        try:             
            if all_events is None:
                all_events = merge_dfs(json_file)
            else:
                all_events = pd.concat([all_events, merge_dfs(json_file)], ignore_index=True)
        
        except IndexError:
            continue
     
columns_to_convert = all_events.columns[~all_events.columns.isin(['matchId',"target"])]   
    
all_events[columns_to_convert] = all_events[columns_to_convert].fillna(0).astype(int) 

In [284]:
all_events_diff = calculate_event_differences(all_events)
all_events_diff_column_names = list(all_events_diff.columns)

all_events_diff_column_names

/home/andres/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


['matchId',
 'jungleMinionsKilled',
 'minionsKilled',
 'totalGold',
 'xp',
 'magicDamageDone',
 'magicDamageDoneToChampions',
 'magicDamageTaken',
 'physicalDamageDone',
 'physicalDamageDoneToChampions',
 'physicalDamageTaken',
 'totalDamageDone',
 'totalDamageDoneToChampions',
 'totalDamageTaken',
 'trueDamageDone',
 'trueDamageDoneToChampions',
 'trueDamageTaken',
 'abilityHaste',
 'abilityPower',
 'armor',
 'armorPen',
 'armorPenPercent',
 'attackDamage',
 'attackSpeed',
 'bonusArmorPenPercent',
 'bonusMagicPenPercent',
 'ccReduction',
 'cooldownReduction',
 'health',
 'healthMax',
 'healthRegen',
 'lifesteal',
 'magicPen',
 'magicPenPercent',
 'magicResist',
 'movementSpeed',
 'omnivamp',
 'physicalVamp',
 'power',
 'powerMax',
 'powerRegen',
 'spellVamp',
 'kills',
 'killType_KILL_FIRST_BLOOD',
 'target']

In [268]:
"""# Get columns that start with 'monsterType'
monster_type_cols = all_events_diff.filter(like='monsterType', axis=1).columns.tolist()

# Get columns that start with 'towerType'
tower_type_cols = all_events_diff.filter(like='towerType', axis=1).columns.tolist()

# Get columns that start with 'killType'
kill_type_cols = all_events_diff.filter(like='killType', axis=1).columns.tolist()

# Combine the columns into a single list
all_cols = monster_type_cols + tower_type_cols + kill_type_cols + ["target","minionsKilled","totalGold"]

all_df = all_events_diff[all_cols]
all_df"""

'# Get columns that start with \'monsterType\'\nmonster_type_cols = all_events_diff.filter(like=\'monsterType\', axis=1).columns.tolist()\n\n# Get columns that start with \'towerType\'\ntower_type_cols = all_events_diff.filter(like=\'towerType\', axis=1).columns.tolist()\n\n# Get columns that start with \'killType\'\nkill_type_cols = all_events_diff.filter(like=\'killType\', axis=1).columns.tolist()\n\n# Combine the columns into a single list\nall_cols = monster_type_cols + tower_type_cols + kill_type_cols + ["target","minionsKilled","totalGold"]\n\nall_df = all_events_diff[all_cols]\nall_df'

In [290]:
target_1row = calculate_event_differences(all_events)
target_1row

/home/andres/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


,matchId,jungleMinionsKilled,minionsKilled,totalGold,xp,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,...,movementSpeed,omnivamp,physicalVamp,power,powerMax,powerRegen,spellVamp,kills,killType_KILL_FIRST_BLOOD,target
0,LA2_1304858815,-12,44,4085,1325,-32207,-2845,2165,43306,5921,...,-83,-2,0,-1414,-1044,-46,0,6,1,1


In [304]:
column_names

['monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'monsterType_AIR_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'towerType_OUTER_TURRET',
 'towerType_INNER_TURRET',
 'towerType_BASE_TURRET',
 'towerType_NEXUS_TURRET',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'killType_KILL_ACE',
 'minionsKilled',
 'totalGold']

In [306]:
# Filtrar el DataFrame target_1row para incluir solo las columnas deseadas y rellenar valores faltantes con 0
filtered_target_1row = target_1row[column_names].fillna(0)

# Preprocesar los datos
df_train_preprocessed, y_train = preprocess(filtered_target_1row)

# Crear DataFrame con los datos preprocesados y los nombres de las columnas
X_train = pd.DataFrame(df_train_preprocessed, columns=column_names)

# Reemplazar valores faltantes por ceros en X_train
X_train = replace_nan_with_zero(X_train)

# Escalar los datos utilizando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)




KeyError: "['monsterType_RIFTHERALD', 'monsterType_WATER_DRAGON', 'monsterType_AIR_DRAGON', 'monsterType_HEXTECH_DRAGON', 'monsterType_FIRE_DRAGON', 'monsterType_CHEMTECH_DRAGON', 'monsterType_EARTH_DRAGON', 'towerType_OUTER_TURRET', 'towerType_INNER_TURRET', 'towerType_BASE_TURRET', 'towerType_NEXUS_TURRET', 'killType_KILL_MULTI', 'killType_KILL_ACE'] not in index"

In [272]:
filtered_df

,monsterType_RIFTHERALD,monsterType_WATER_DRAGON,monsterType_AIR_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_CHEMTECH_DRAGON,monsterType_EARTH_DRAGON,towerType_OUTER_TURRET,towerType_INNER_TURRET,towerType_BASE_TURRET,towerType_NEXUS_TURRET,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,killType_KILL_ACE,minionsKilled,totalGold
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,44,4085


In [245]:
X_train

,monsterType_RIFTHERALD,monsterType_WATER_DRAGON,monsterType_AIR_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_CHEMTECH_DRAGON,monsterType_EARTH_DRAGON,towerType_OUTER_TURRET,towerType_INNER_TURRET,towerType_BASE_TURRET,towerType_NEXUS_TURRET,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,killType_KILL_ACE,minionsKilled,totalGold
30304,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.390244,-0.031090
38329,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.048780,0.468001
24646,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.024390,0.001323
15350,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.195122,-0.429965
22919,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.195122,1.022656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.243902,-0.270217
44732,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.439024,-0.164710
38158,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,-0.780488,-0.850670
860,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-0.439024,0.652886
